In [1]:
from transformers import BertTokenizer, TFBertModel
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('imdb_cleaned.csv')

In [4]:
df.head()

,Unnamed: 0,review_text,sentiment_score,review_stopwords_noise_remove
0,0,one of the other reviewers has mentioned that ...,positive,one reviewer mention watch 1 episode hook righ...
1,1,a wonderful little production. <br /><br />the...,positive,wonderful little production film technique una...
2,2,i thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...
3,3,basically there's a family where a little boy ...,negative,basically there family little boy jake think t...
4,4,"petter mattei's ""love in the time of money"" is...",positive,petter love time money visually stunning film ...


In [5]:
batch_size = 100 
all_embeddings = []
all_labels = []

for i in range(0, len(df['review_stopwords_noise_remove']), batch_size):
    batch_data = tokenizer.batch_encode_plus(
        df['review_stopwords_noise_remove'][i:i + batch_size],
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    output = model(**batch_data)
    po = output.pooler_output

    all_embeddings.append(po)
    all_labels.append(df['sentiment_score'][i:i + batch_size])

In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
del df

In [8]:
x=tf.concat(all_embeddings,axis=0).numpy()
y=tf.concat(all_labels,axis=0).numpy()
y=np.where(y=='positive',1,0)

In [9]:
x.shape

(50000, 768)

In [10]:
y.shape

(50000,)

In [11]:
#y = y.reshape(-1, 1)

In [12]:
del all_embeddings
del all_labels

In [13]:
x_tr,x_te,y_tr,y_te=train_test_split(x,y,test_size=0.3,random_state=0)

In [14]:
del x,y

In [15]:
x_tr.shape

(35000, 768)

In [16]:
x_te.shape

(15000, 768)

In [17]:
y_tr.shape

(35000,)

In [18]:
y_te.shape

(15000,)

In [20]:
model_ann=tf.keras.models.Sequential()
model_ann.add(tf.keras.layers.Dense(units=1000,activation='relu'))
model_ann.add(tf.keras.layers.Dense(units=1000,activation='relu'))
model_ann.add(tf.keras.layers.Dense(units=1000,activation='relu'))
model_ann.add(tf.keras.layers.Dense(units=1000,activation='relu'))
model_ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
model_ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model_ann.fit(x_tr,y_tr,batch_size=10,epochs=5)


Epoch 1/5


3500/3500 [==============================] - 97s 27ms/step - loss: 1.7719e-04 - accuracy: 1.0000
Epoch 2/5
3500/3500 [==============================] - 94s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
3500/3500 [==============================] - 94s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
3500/3500 [==============================] - 94s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
3500/3500 [==============================] - 94s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [21]:
del x_tr,y_tr

In [22]:
y_pred=model_ann.predict(x_te)

469/469 [==============================] - 3s 7ms/step


In [23]:
y_pred.shape

(15000, 1)

In [24]:
y_te.shape

(15000,)

In [25]:
#a=y_pred.reshape(-1,1)

In [26]:
#a.shape

(15000, 1)

In [27]:
y_pred.shape

(15000, 1)

In [28]:
len(y_pred)

15000

In [29]:
y_pred=(y_pred>0.5).astype(int)

In [30]:
y_te

array([0, 0, 0, ..., 0, 0, 0])

In [31]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [32]:
y_pred.shape

(15000, 1)

In [33]:
print("Accuracy score is",accuracy_score(y_te,y_pred)*100)

Accuracy score is 100.0
